## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now() - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=6)

def format_date(date):
    day = date.day
    # Months 3-7 (March–July) should be full; all others abbreviated with a period
    if date.month in (3, 4, 5, 6, 7):
        month_str = date.strftime("%B")      # e.g. "March", "April", ...
    else:
        month_str = date.strftime("%b") + "."  # e.g. "Jan.", "Feb.", "Aug.", ...

    return f"{month_str} {day}"


# Format both dates
week_ending = format_date(today)
week_starting = format_date(one_week_ago)

print(week_starting, week_ending)

Aug. 18 Aug. 24


In [3]:
filename = glob.glob('*redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df.sort_values(by='PRICE',ascending=False)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING),SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
99,PAST SALE,August-21-2025,Condo/Co-op,17749 Collins Ave #902,Sunny Isles Beach,FL,33160.0,6925000.0,4.0,6.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11637117,N,Y,25.940064,-80.120313
2,PAST SALE,August-18-2025,Condo/Co-op,785 Crandon Blvd #1803,Key Biscayne,FL,33149.0,4025000.0,3.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/785-Cra...,MARMLS,A11807631,N,Y,25.687534,-80.159408
40,PAST SALE,August-20-2025,Condo/Co-op,900 Brickell Key Blvd #2104,Miami,FL,33131.0,2400000.0,3.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/900-Brickell-K...,MARMLS,A11795493,N,Y,25.769263,-80.185852
28,PAST SALE,August-20-2025,Condo/Co-op,3300 NE 188th St Unit UPH15 + ROOF TOP,Aventura,FL,33180.0,2200000.0,3.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/3300-NE-188...,MARMLS,A11785525,N,Y,25.948958,-80.136544
68,PAST SALE,August-18-2025,Condo/Co-op,125 Ocean Dr Unit L0101,Miami Beach,FL,33139.0,2090000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/125-Ocea...,MARMLS,A11770568,N,Y,25.770052,-80.132576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,PAST SALE,August-22-2025,Condo/Co-op,1515 NE 125th Ter #211,North Miami,FL,33161.0,157500.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami/1515-NE-...,MARMLS,A11741118,N,Y,25.892154,-80.167845
54,PAST SALE,August-21-2025,Condo/Co-op,1500 NE 127th St #114,North Miami,FL,33161.0,128000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami/1500-NE-...,MARMLS,A11788194,N,Y,25.892481,-80.168117
87,PAST SALE,August-21-2025,Condo/Co-op,18900 NE 14th Ave Unit D204,Miami,FL,33179.0,127500.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/18900-NE-14th-...,MARMLS,A11702014,N,Y,25.950054,-80.173451
15,PAST SALE,August-18-2025,Condo/Co-op,11111 Biscayne Blvd #2100,Miami,FL,33181.0,100000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/11111-Biscayne...,MARMLS,A11836542,N,Y,25.879253,-80.165007


In [5]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [6]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [7]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [8]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,August-18-2025,Condo/Co-op,2450 NE 135th St #406,North Miami,FL,33181.0,200000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami/2450-NE-...,MARMLS,A11723850,N,Y,25.899965,-80.14936


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [9]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [10]:
df2 = df[(df['SOLD DATE'] >= one_week_ago) & (df['SOLD DATE'] < today)]

In [11]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-08-20    23
2025-08-22    21
2025-08-19    16
2025-08-21    16
Name: count, dtype: int64

In [12]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [13]:
df_top_ten = df2.head(10)

In [14]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
99,PAST SALE,2025-08-21,Condo/Co-op,17749 Collins Ave #902,Sunny Isles Beach,FL,33160.0,6925000.0,4.0,6.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11637117,N,Y,25.940064,-80.120313
40,PAST SALE,2025-08-20,Condo/Co-op,900 Brickell Key Blvd #2104,Miami,FL,33131.0,2400000.0,3.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/900-Brickell-K...,MARMLS,A11795493,N,Y,25.769263,-80.185852
28,PAST SALE,2025-08-20,Condo/Co-op,3300 NE 188th St Unit UPH15 + ROOF TOP,Aventura,FL,33180.0,2200000.0,3.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/3300-NE-188...,MARMLS,A11785525,N,Y,25.948958,-80.136544
88,PAST SALE,2025-08-22,Condo/Co-op,18911 Collins Ave #1104,Sunny Isles Beach,FL,33160.0,1585000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11743925,N,Y,25.950878,-80.119958
81,PAST SALE,2025-08-20,Condo/Co-op,2901 S Bayshore Dr Unit 13F,Miami,FL,33133.0,1575000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2901-S-Bayshor...,MARMLS,A11735043,N,Y,25.727753,-80.239494
50,PAST SALE,2025-08-20,Condo/Co-op,1010 Brickell Ave #3501,Miami,FL,33131.0,1575000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1010-Brickell-...,MARMLS,A11796532,N,Y,25.764203,-80.192181
51,PAST SALE,2025-08-20,Condo/Co-op,9 Island Ave #814,Miami Beach,FL,33139.0,1542500.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/9-Island...,MARMLS,A11797596,N,Y,25.790337,-80.148773
72,PAST SALE,2025-08-22,Condo/Co-op,5875 Collins Ave #1603,Miami Beach,FL,33140.0,1475000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5875-Col...,MARMLS,A11765395,N,Y,25.840891,-80.120220
63,PAST SALE,2025-08-19,Condo/Co-op,851 NE 1st Ave #4502,Miami,FL,33132.0,1465000.0,3.0,4.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/851-NE-1st-Ave...,MARMLS,A11787570,N,Y,25.782453,-80.191836
42,PAST SALE,2025-08-19,Condo/Co-op,21200 Point Pl #1802,Aventura,FL,33180.0,1300000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/21200-Point...,MARMLS,A11806655,N,Y,25.973121,-80.125125


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [15]:
def process_response(url):
    try:
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"
        querystring = {"url": url}
        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)
        time.sleep(0.2)
        response.raise_for_status()

        # If .json() returns None, fall back on empty dict
        data = response.json() or {}

        # Drill down with or {} at each step to avoid None.get errors
        panel_info = (data.get('data') or {}) \
            .get('mainHouseInfoPanelInfo') or {}
        main_info = panel_info.get('mainHouseInfo') or {}

        # Now safe to pull lists
        listing_agents = main_info.get('listingAgents') or []
        buying_agents  = main_info.get('buyingAgents')  or []

        # Defaults
        list_agent_name_1 = list_broker_name_1 = None
        list_agent_name_2 = list_broker_name_2 = None
        buy_agent_name_1  = buy_broker_name_1  = None
        buy_agent_name_2  = buy_broker_name_2  = None

        # Extract up to two listing agents
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}) \
                                             .get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}) \
                                             .get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')

        # Extract up to two buying agents
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}) \
                                            .get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}) \
                                            .get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')

        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }


In [16]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [17]:
response_df = pd.DataFrame(response_list)

In [18]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [19]:
df_top_ten = merged_df

## Current Week's Values

In [20]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

76


In [21]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$644,907


In [22]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$493


In [23]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$49,012,900


In [24]:
def load_last_week_data_df(filename='last_week_data.csv'):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return None
    
last_week = load_last_week_data_df()

In [25]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
# previous_week_condo_sales = input()
previous_week_condo_sales = last_week['Condo Sales'].iloc[0]
print(previous_week_condo_sales)
print('-----------')

print('Input Previous Week Condo Average Sales Price:')
# previous_week_average_sales_price = input()
previous_week_average_sales_price = last_week['Average Sale Price'].iloc[0]
print(previous_week_average_sales_price)
print('-----------')

print('Input Previous Week Condo Average PSF:')
# previous_week_average_psf = input()
previous_week_average_psf = last_week['Average PSF'].iloc[0]
print(previous_week_average_psf)
print('-----------')

print('Input Previous Week Condo Sales total (ex: 198_000_000)')
# previous_week_condo_sales_total = input()
previous_week_condo_sales_total = last_week['Last Week Total Dollar Volume'].iloc[0]
print(previous_week_condo_sales_total)


## Clean Data

Input county name:


 Miami-Dade


Input Previous Week Condo Sales Total (number units sold):
55
-----------
Input Previous Week Condo Average Sales Price:
1713245
-----------
Input Previous Week Condo Average PSF:
820
-----------
Input Previous Week Condo Sales total (ex: 198_000_000)
94200000


In [26]:
last_week

,Condo Sales,Average Sale Price,Average PSF,Last Week Total Dollar Volume
0,55,1713245,820,94200000


In [27]:
last_week['Last Week Total Dollar Volume'].iloc[0]

94200000

In [29]:
def save_current_week_data_df(df, filename='last_week_data.csv'):
    df.to_csv(filename, index=False)

# Create a dictionary of your current week data.
current_week_data = {
    "Condo Sales": current_week_condo_sales,
    "Average Sale Price": average_sale_price,
    "Average PSF": average_price_psf,
    "Last Week Total Dollar Volume": current_week_condo_sales_total
}

# Convert the dictionary into a DataFrame.
current_week_summary_df = pd.DataFrame([current_week_data])

# Save the DataFrame to a CSV file.
save_current_week_data_df(current_week_summary_df)

# Next week, load last week's data automatically.
last_week_df = load_last_week_data_df()
if last_week_df is not None:
    print("Last week's data:")
    print(last_week_df)
else:
    print("No previous data found.")

Last week's data:
   Condo Sales  Average Sale Price  Average PSF  Last Week Total Dollar Volume
0           76              644907          493                     49012900.0


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [30]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [31]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [32]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [33]:
pd.set_option('display.max_columns',None)

In [34]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-08-21,Condo/Co-op,17749 Collins Ave #902,Sunny Isles Beach,FL,33160.0,6925000.0,4.0,6.5,17749 COLLINS AVENUE COND,4609.0,NaN,2015.0,NaN,1502.0,11166.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11637117,N,Y,25.940064,-80.120313,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Maria Barbosa,Elite International Realty Inc,None,None,Marvin Anhalt,Anhalt Realty,None,None,1,orange


In [35]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [36]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-08-21,Condo/Co-op,17749 Collins Ave #902,Sunny Isles Beach,FL,33160.0,6925000.0,4.0,6.5,17749 COLLINS AVENUE COND,4609.0,NaN,2015.0,NaN,1502.0,11166.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11637117,N,Y,25.940064,-80.120313,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Maria Barbosa,Elite International Realty Inc,None,None,Marvin Anhalt,Anhalt Realty,None,None,1,orange
1,PAST SALE,2025-08-20,Condo/Co-op,900 Brickell Key Blvd #2104,Miami,FL,33131.0,2400000.0,3.0,4.5,ASIA CONDO,2631.0,NaN,2008.0,NaN,912.0,4728.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/900-Brickell-K...,MARMLS,A11795493,N,Y,25.769263,-80.185852,https://www.redfin.com/FL/Miami/900-Brickell-K...,Christopher Jude,"PGutman, LLC",None,None,Daniel Hernandez,"Compass Florida, LLC",None,None,2,blue
2,PAST SALE,2025-08-20,Condo/Co-op,3300 NE 188th St Unit UPH15 + ROOF TOP,Aventura,FL,33180.0,2200000.0,3.0,4.5,ECHO CONDO,2962.0,NaN,2015.0,NaN,743.0,4141.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/3300-NE-188...,MARMLS,A11785525,N,Y,25.948958,-80.136544,https://www.redfin.com/FL/Aventura/3300-NE-188...,Uri Sefchovich,The Keyes Company,None,None,Uri Sefchovich,The Keyes Company,None,None,3,blue
3,PAST SALE,2025-08-22,Condo/Co-op,18911 Collins Ave #1104,Sunny Isles Beach,FL,33160.0,1585000.0,2.0,2.5,OCEAN THREE CONDO,1903.0,NaN,2003.0,NaN,833.0,1619.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11743925,N,Y,25.950878,-80.119958,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Lana Levitina,Coldwell Banker Realty,None,None,Oswaldo Sanchez Robleno,Miami New Realty,None,None,4,blue
4,PAST SALE,2025-08-20,Condo/Co-op,2901 S Bayshore Dr Unit 13F,Miami,FL,33133.0,1575000.0,3.0,2.0,YACHT HARBOUR CONDO,1587.0,NaN,1975.0,NaN,992.0,1985.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2901-S-Bayshor...,MARMLS,A11735043,N,Y,25.727753,-80.239494,https://www.redfin.com/FL/Miami/2901-S-Bayshor...,Allison Blumenthal,Brown Harris Stevens,None,None,Monica Cardoso,EXP Realty South LLC,None,None,5,blue
5,PAST SALE,2025-08-20,Condo/Co-op,1010 Brickell Ave #3501,Miami,FL,33131.0,1575000.0,3.0,3.0,1010 BRICKELL CONDO,1753.0,NaN,2017.0,NaN,898.0,2332.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1010-Brickell-...,MARMLS,A11796532,N,Y,25.764203,-80.192181,https://www.redfin.com/FL/Miami/1010-Brickell-...,David Siddons,Douglas Elliman,Shakira Sanchez,Douglas Elliman,Laura Preuss Kuhne,Douglas Elliman,None,None,6,blue
6,PAST SALE,2025-08-20,Condo/Co-op,9 Island Ave #814,Miami Beach,FL,33139.0,1542500.0,3.0,3.0,NINE ISLAND AVENUE CONDO,2388.0,NaN,1981.0,NaN,646.0,2991.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/9-Island...,MARMLS,A11797596,N,Y,25.790337,-80.148773,https://www.redfin.com/FL/Miami-Beach/9-Island...,Carol Curtis,First Pro International Rlty,None,None,Carol Curtis,First Pro International Rlty,None,None,7,blue
7,PAST SALE,2025-08-22,Condo/Co-op,5875 Collins Ave #1603,Miami Beach,FL,33140.0,1475000.0,2.0,2.5,MEI CONDO,1094.0,NaN,2008.0,NaN,1348.0,2758.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5875-Col...,MARMLS,A11765395,N,Y,25.840891,-80.120220,https://www.redfin.com/FL/Miami-Beach/5875-Col...,Armando Javier,Jasal Int'l Realty,Silvia Rivera,Jasal Int'l Realty,Christopher King,"Compass Florida, LLC",None,None,8,blue
8,PAST SALE,2025-08-19,Condo/Co-op,851 NE 1st Ave #4502,Miami,FL,33132.0,1465000.0,3.0,4.0,PARAMOUNT MIAMI WORLDCENT,2164.0,NaN,2019.0,NaN,677.0,3816.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/851-NE-1st-Ave...,MARMLS,A1

In [37]:
%store -r map_box_api_key

In [38]:
import folium
from branca.element import Template, MacroElement

# ——— Your title & caption HTML ———
title_html = f'''
  <h3 align="center" style="font-size:16px"><b>Recent {county} County Condo Sales</b></h3>
'''
caption_html = f'''
  <p align="center" style="font-size:13px"><i>{week_starting} — {week_ending}</i></p>
'''

# ——— Base map ———
m = folium.Map(
    location=df[['LATITUDE','LONGITUDE']].mean().to_list(),
    zoom_start=10.5,
    tiles=None
)

# ——— Tile layers ———
folium.TileLayer('OpenStreetMap', name='OSM', control=False).add_to(m)
folium.TileLayer(
    tiles=(
        "https://api.mapbox.com/styles/v1/"
        "mapbox/streets-v11/tiles/256/{z}/{x}/{y}@2x"
        f"?access_token={map_box_api_key}"
    ),
    attr="Mapbox", name="Streets", control=False
).add_to(m)

# ——— Add your markers ———
for _, row in df_top_ten.iterrows():
    lat, lon = row['LATITUDE'], row['LONGITUDE']
    popup = folium.Popup(popup_html(row), max_width=400)

    if row['COLOR'] == 'blue':
        # Regular sale: blue circle
        folium.CircleMarker(
            location=(lat, lon),
            radius=7,
            fill=True,
            fill_color='blue',
            fill_opacity=0.5,
            color=None,
            popup=popup
        ).add_to(m)
    else:
        # Top sale: gold star
        folium.Marker(
            location=(lat, lon),
            popup=popup,
            icon=folium.DivIcon(
                html="""
                <div style="font-size:24px; line-height:24px; color:gold;">
                  &#9733;
                </div>
                """
            )
        ).add_to(m)

# ——— Layer control, title & caption ———
folium.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

# ——— Optional legend via branca ———
legend_html = """
{% macro html(this, kwargs) %}
  <div style="
      position: fixed;
      bottom: 50px; left: 50px;
      background: white; padding: 8px;
      border: 2px solid grey;
      font-size: 12px;
      z-index: 9999;
  ">
    <div style="display:inline-block; font-size:18px; color:gold; margin-right:6px;">&#9733;</div> Top Sale<br>
    <i style="background: blue; width: 10px; height: 10px;
       display: inline-block; margin-right: 6px;"></i> All other sales
  </div>
{% endmacro %}
"""
legend = MacroElement()
legend._template = Template(legend_html)
m.get_root().add_child(legend)

# ——— Display ———
m


In [39]:
m.save('index.html')

## Data snagger

In [40]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [41]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-08-21,Condo/Co-op,17749 Collins Ave #902,Sunny Isles Beach,FL,33160.0,6925000.0,4.0,6.5,17749 COLLINS AVENUE COND,4609.0,NaN,2015.0,NaN,1502.0,11166.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11637117,N,Y,25.940064,-80.120313,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Maria Barbosa,Elite International Realty Inc,None,None,Marvin Anhalt,Anhalt Realty,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [42]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MiamiDade_County_condo_sales_week_ending_08252025


# CREATE TEMPLATE 

In [43]:
muni_set = set(df_top_ten['CITY'])

In [44]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [45]:
df_top_ten.reset_index(inplace=True,drop=True)

In [46]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [47]:
top_sale

'17749 COLLINS AVENUE COND at 17749 Collins Ave #902 in Sunny Isles Beach'

In [48]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [49]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [50]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-08-21,Condo/Co-op,17749 Collins Ave #902,Sunny Isles Beach,FL,33160.0,6925000.0,4.0,6.5,17749 COLLINS AVENUE COND,4609.0,NaN,2015.0,NaN,1502.0,11166.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11637117,N,Y,25.940064,-80.120313,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Maria Barbosa,Elite International Realty Inc,None,None,Marvin Anhalt,Anhalt Realty,None,None,1,orange,17749 COLLINS AVENUE COND at 17749 Collins Ave...
1,PAST SALE,2025-08-20,Condo/Co-op,900 Brickell Key Blvd #2104,Miami,FL,33131.0,2400000.0,3.0,4.5,ASIA CONDO,2631.0,NaN,2008.0,NaN,912.0,4728.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/900-Brickell-K...,MARMLS,A11795493,N,Y,25.769263,-80.185852,https://www.redfin.com/FL/Miami/900-Brickell-K...,Christopher Jude,"PGutman, LLC",None,None,Daniel Hernandez,"Compass Florida, LLC",None,None,2,blue,ASIA CONDO at 900 Brickell Key Blvd #2104 in M...
2,PAST SALE,2025-08-20,Condo/Co-op,3300 NE 188th St Unit UPH15 + ROOF TOP,Aventura,FL,33180.0,2200000.0,3.0,4.5,ECHO CONDO,2962.0,NaN,2015.0,NaN,743.0,4141.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/3300-NE-188...,MARMLS,A11785525,N,Y,25.948958,-80.136544,https://www.redfin.com/FL/Aventura/3300-NE-188...,Uri Sefchovich,The Keyes Company,None,None,Uri Sefchovich,The Keyes Company,None,None,3,blue,ECHO CONDO at 3300 NE 188th St Unit UPH15 + RO...
3,PAST SALE,2025-08-22,Condo/Co-op,18911 Collins Ave #1104,Sunny Isles Beach,FL,33160.0,1585000.0,2.0,2.5,OCEAN THREE CONDO,1903.0,NaN,2003.0,NaN,833.0,1619.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11743925,N,Y,25.950878,-80.119958,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Lana Levitina,Coldwell Banker Realty,None,None,Oswaldo Sanchez Robleno,Miami New Realty,None,None,4,blue,OCEAN THREE CONDO at 18911 Collins Ave #1104 i...
4,PAST SALE,2025-08-20,Condo/Co-op,2901 S Bayshore Dr Unit 13F,Miami,FL,33133.0,1575000.0,3.0,2.0,YACHT HARBOUR CONDO,1587.0,NaN,1975.0,NaN,992.0,1985.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2901-S-Bayshor...,MARMLS,A11735043,N,Y,25.727753,-80.239494,https://www.redfin.com/FL/Miami/2901-S-Bayshor...,Allison Blumenthal,Brown Harris Stevens,None,None,Monica Cardoso,EXP Realty South LLC,None,None,5,blue,YACHT HARBOUR CONDO at 2901 S Bayshore Dr Unit...
5,PAST SALE,2025-08-20,Condo/Co-op,1010 Brickell Ave #3501,Miami,FL,33131.0,1575000.0,3.0,3.0,1010 BRICKELL CONDO,1753.0,NaN,2017.0,NaN,898.0,2332.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1010-Brickell-...,MARMLS,A11796532,N,Y,25.764203,-80.192181,https://www.redfin.com/FL/Miami/1010-Brickell-...,David Siddons,Douglas Elliman,Shakira Sanchez,Douglas Elliman,Laura Preuss Kuhne,Douglas Elliman,None,None,6,blue,1010 BRICKELL CONDO at 1010 Brickell Ave #3501...
6,PAST SALE,2025-08-20,Condo/Co-op,9 Island Ave #814,Miami Beach,FL,33139.0,1542500.0,3.0,3.0,NINE ISLAND AVENUE CONDO,2388.0,NaN,1981.0,NaN,646.0,2991.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/9-Island...,MARMLS,A11797596,N,Y,25.790337,-80.148773,https://www.redfin.com/FL/Miami-Beach/9-Island...,Carol Curtis,First Pro International Rlty,None,None,Carol Curtis,First Pro International Rlty,None,None,7,blue,NINE ISLAND AVENUE CONDO at 9 Island Ave #814 ...
7,PAST SALE,2025-08-22,Condo/Co-op,5875 Collins Ave #1603,Miami Beach,FL,33140.0,1475000.0,2.0,2.5,MEI CONDO,1094.0,NaN,2008.0,NaN,1348.0,2758.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5875-Col...,MARMLS,A11765395,N,Y,25.840891,-80.120220,https://www.redfin.com/FL/Miami-Beach/

In [51]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}, condos, condo market

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: 17749 COLLINS AVENUE COND closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,300,000 to $6,925,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Miami Beach, Sunny Isles Beach, Aventura, Miami, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 76 condo sales totaling $49,012,900 million from Aug. 18 to Aug. 24. The previous week, brokers closed 55 condo sales totaling $94,200,000.

Last week’s units sold for an average of $644,907, lower than the $1,713,245 average price from the pr

In [52]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [53]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [54]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [55]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-08-21,Condo/Co-op,17749 Collins Ave #902,Sunny Isles Beach,FL,33160.0,6925000.0,4.0,6.5,17749 COLLINS AVENUE COND,4609.0,NaN,2015.0,NaN,1502.0,11166.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11637117,N,Y,25.940064,-80.120313,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Maria Barbosa,Elite International Realty Inc,None,None,Marvin Anhalt,Anhalt Realty,None,None,1,orange,17749 COLLINS AVENUE COND at 17749 Collins Ave...
1,PAST SALE,2025-08-20,Condo/Co-op,900 Brickell Key Blvd #2104,Miami,FL,33131.0,2400000.0,3.0,4.5,ASIA CONDO,2631.0,NaN,2008.0,NaN,912.0,4728.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/900-Brickell-K...,MARMLS,A11795493,N,Y,25.769263,-80.185852,https://www.redfin.com/FL/Miami/900-Brickell-K...,Christopher Jude,"PGutman, LLC",None,None,Daniel Hernandez,"Compass Florida, LLC",None,None,2,blue,ASIA CONDO at 900 Brickell Key Blvd #2104 in M...
2,PAST SALE,2025-08-20,Condo/Co-op,3300 NE 188th St Unit UPH15 + ROOF TOP,Aventura,FL,33180.0,2200000.0,3.0,4.5,ECHO CONDO,2962.0,NaN,2015.0,NaN,743.0,4141.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/3300-NE-188...,MARMLS,A11785525,N,Y,25.948958,-80.136544,https://www.redfin.com/FL/Aventura/3300-NE-188...,Uri Sefchovich,The Keyes Company,None,None,Uri Sefchovich,The Keyes Company,None,None,3,blue,ECHO CONDO at 3300 NE 188th St Unit UPH15 + RO...
3,PAST SALE,2025-08-22,Condo/Co-op,18911 Collins Ave #1104,Sunny Isles Beach,FL,33160.0,1585000.0,2.0,2.5,OCEAN THREE CONDO,1903.0,NaN,2003.0,NaN,833.0,1619.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11743925,N,Y,25.950878,-80.119958,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Lana Levitina,Coldwell Banker Realty,None,None,Oswaldo Sanchez Robleno,Miami New Realty,None,None,4,blue,OCEAN THREE CONDO at 18911 Collins Ave #1104 i...
4,PAST SALE,2025-08-20,Condo/Co-op,2901 S Bayshore Dr Unit 13F,Miami,FL,33133.0,1575000.0,3.0,2.0,YACHT HARBOUR CONDO,1587.0,NaN,1975.0,NaN,992.0,1985.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2901-S-Bayshor...,MARMLS,A11735043,N,Y,25.727753,-80.239494,https://www.redfin.com/FL/Miami/2901-S-Bayshor...,Allison Blumenthal,Brown Harris Stevens,None,None,Monica Cardoso,EXP Realty South LLC,None,None,5,blue,YACHT HARBOUR CONDO at 2901 S Bayshore Dr Unit...
5,PAST SALE,2025-08-20,Condo/Co-op,1010 Brickell Ave #3501,Miami,FL,33131.0,1575000.0,3.0,3.0,1010 BRICKELL CONDO,1753.0,NaN,2017.0,NaN,898.0,2332.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1010-Brickell-...,MARMLS,A11796532,N,Y,25.764203,-80.192181,https://www.redfin.com/FL/Miami/1010-Brickell-...,David Siddons,Douglas Elliman,Shakira Sanchez,Douglas Elliman,Laura Preuss Kuhne,Douglas Elliman,None,None,6,blue,1010 BRICKELL CONDO at 1010 Brickell Ave #3501...
6,PAST SALE,2025-08-20,Condo/Co-op,9 Island Ave #814,Miami Beach,FL,33139.0,1542500.0,3.0,3.0,NINE ISLAND AVENUE CONDO,2388.0,NaN,1981.0,NaN,646.0,2991.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/9-Island...,MARMLS,A11797596,N,Y,25.790337,-80.148773,https://www.redfin.com/FL/Miami-Beach/9-Island...,Carol Curtis,First Pro International Rlty,None,None,Carol Curtis,First Pro International Rlty,None,None,7,blue,NINE ISLAND AVENUE CONDO at 9 Island Ave #814 ...
7,PAST SALE,2025-08-22,Condo/Co-op,5875 Collins Ave #1603,Miami Beach,FL,33140.0,1475000.0,2.0,2.5,MEI CONDO,1094.0,NaN,2008.0,NaN,1348.0,2758.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5875-Col...,MARMLS,A11765395,N,Y,25.840891,-80.120220,https://www.redfin.com/FL/Miami-Beach/

In [56]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: 17749 COLLINS AVENUE COND closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,300,000 to $6,925,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Miami Beach, Sunny Isles Beach, Aventura, Miami, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 76 condo sales totaling $49,012,900 million from Aug. 18 to Aug. 24. The previous week, brokers closed 55 condo sales totaling $94,200,000.

Last week’s units sold for an average of $644,907, lower than the $1,713,245 average price from the pr

In [64]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Sunny-Isles-Beach/17749-Collins-Ave-33160/unit-902/home/110130545


In [65]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Miami/900-Brickell-Key-Blvd-33131/unit-2104/home/43411812


In [66]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Aventura/21200-Point-Pl-33180/unit-1802/home/192358911


In [67]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Miami/851-NE-1st-Ave-33132/unit-4502/home/170057071


In [68]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Sunny-Isles-Beach/17749-Collins-Ave-33160/unit-902/home/110130545


In [69]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Aventura/21200-Point-Pl-33180/unit-1802/home/192358911


In [70]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-08-21,Condo/Co-op,17749 Collins Ave #902,Sunny Isles Beach,FL,33160.0,6925000.0,4.0,6.5,17749 COLLINS AVENUE COND,4609.0,NaN,2015.0,NaN,1502.0,11166.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11637117,N,Y,25.940064,-80.120313,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Maria Barbosa,Elite International Realty Inc,None,None,Marvin Anhalt,Anhalt Realty,None,None,1,orange,17749 COLLINS AVENUE COND at 17749 Collins Ave...
1,PAST SALE,2025-08-20,Condo/Co-op,900 Brickell Key Blvd #2104,Miami,FL,33131.0,2400000.0,3.0,4.5,ASIA CONDO,2631.0,NaN,2008.0,NaN,912.0,4728.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/900-Brickell-K...,MARMLS,A11795493,N,Y,25.769263,-80.185852,https://www.redfin.com/FL/Miami/900-Brickell-K...,Christopher Jude,"PGutman, LLC",None,None,Daniel Hernandez,"Compass Florida, LLC",None,None,2,blue,ASIA CONDO at 900 Brickell Key Blvd #2104 in M...
2,PAST SALE,2025-08-20,Condo/Co-op,3300 NE 188th St Unit UPH15 + ROOF TOP,Aventura,FL,33180.0,2200000.0,3.0,4.5,ECHO CONDO,2962.0,NaN,2015.0,NaN,743.0,4141.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/3300-NE-188...,MARMLS,A11785525,N,Y,25.948958,-80.136544,https://www.redfin.com/FL/Aventura/3300-NE-188...,Uri Sefchovich,The Keyes Company,None,None,Uri Sefchovich,The Keyes Company,None,None,3,blue,ECHO CONDO at 3300 NE 188th St Unit UPH15 + RO...
3,PAST SALE,2025-08-22,Condo/Co-op,18911 Collins Ave #1104,Sunny Isles Beach,FL,33160.0,1585000.0,2.0,2.5,OCEAN THREE CONDO,1903.0,NaN,2003.0,NaN,833.0,1619.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11743925,N,Y,25.950878,-80.119958,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Lana Levitina,Coldwell Banker Realty,None,None,Oswaldo Sanchez Robleno,Miami New Realty,None,None,4,blue,OCEAN THREE CONDO at 18911 Collins Ave #1104 i...
4,PAST SALE,2025-08-20,Condo/Co-op,2901 S Bayshore Dr Unit 13F,Miami,FL,33133.0,1575000.0,3.0,2.0,YACHT HARBOUR CONDO,1587.0,NaN,1975.0,NaN,992.0,1985.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2901-S-Bayshor...,MARMLS,A11735043,N,Y,25.727753,-80.239494,https://www.redfin.com/FL/Miami/2901-S-Bayshor...,Allison Blumenthal,Brown Harris Stevens,None,None,Monica Cardoso,EXP Realty South LLC,None,None,5,blue,YACHT HARBOUR CONDO at 2901 S Bayshore Dr Unit...
5,PAST SALE,2025-08-20,Condo/Co-op,1010 Brickell Ave #3501,Miami,FL,33131.0,1575000.0,3.0,3.0,1010 BRICKELL CONDO,1753.0,NaN,2017.0,NaN,898.0,2332.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1010-Brickell-...,MARMLS,A11796532,N,Y,25.764203,-80.192181,https://www.redfin.com/FL/Miami/1010-Brickell-...,David Siddons,Douglas Elliman,Shakira Sanchez,Douglas Elliman,Laura Preuss Kuhne,Douglas Elliman,None,None,6,blue,1010 BRICKELL CONDO at 1010 Brickell Ave #3501...
6,PAST SALE,2025-08-20,Condo/Co-op,9 Island Ave #814,Miami Beach,FL,33139.0,1542500.0,3.0,3.0,NINE ISLAND AVENUE CONDO,2388.0,NaN,1981.0,NaN,646.0,2991.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/9-Island...,MARMLS,A11797596,N,Y,25.790337,-80.148773,https://www.redfin.com/FL/Miami-Beach/9-Island...,Carol Curtis,First Pro International Rlty,None,None,Carol Curtis,First Pro International Rlty,None,None,7,blue,NINE ISLAND AVENUE CONDO at 9 Island Ave #814 ...
7,PAST SALE,2025-08-22,Condo/Co-op,5875 Collins Ave #1603,Miami Beach,FL,33140.0,1475000.0,2.0,2.5,MEI CONDO,1094.0,NaN,2008.0,NaN,1348.0,2758.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5875-Col...,MARMLS,A11765395,N,Y,25.840891,-80.120220,https://www.redfin.com/FL/Miami-Beach/

## Time on Market Calculator

In [71]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2025, 5, 2) ## List (Earlier) date
date2 = datetime(2025, 8, 20) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

110


## Clean CSV for Datawrapper Chart

In [72]:
chart_df = df_top_ten

In [73]:
chart_df = chart_df.fillna(" ")

In [74]:
chart_df['LISTING AGENTS'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING AGENTS'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [75]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [76]:
chart_df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME', 'LISTING AGENTS',
       'BUYING AGENTS'],
      dtype='object')

In [77]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "BEDS", 
    "BATHS",
    "SOURCE", 
    "MLS#"
]

In [78]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [79]:
chart_df = chart_df.rename(columns={'$/SQUARE FEET':'PPSF'})

In [80]:
chart_df['PPSF'].astype(float)

0    1502.0
1     912.0
2     743.0
3     833.0
4     992.0
5     898.0
6     646.0
7    1348.0
8     677.0
9     476.0
Name: PPSF, dtype: float64

In [81]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['PPSF'] = chart_df['PPSF'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))

In [82]:
desired_order = [
    'SOLD DATE',
    'LOCATION',
    'ADDRESS',
    'PRICE',
    'SQUARE FEET',
    'PPSF',
    'YEAR BUILT',
    'LISTING AGENTS',
    'BUYING AGENTS'
]

chart_df = chart_df[desired_order]


In [83]:
chart_df

,SOLD DATE,LOCATION,ADDRESS,PRICE,SQUARE FEET,PPSF,YEAR BUILT,LISTING AGENTS,BUYING AGENTS
0,2025-08-21,17749 COLLINS AVENUE COND,17749 Collins Ave #902 in Sunny Isles Beach,"$6,925,000","4,609","$1,502",2015,Maria Barbosa Elite International Realty Inc,Marvin Anhalt Anhalt Realty
1,2025-08-20,ASIA CONDO,900 Brickell Key Blvd #2104 in Miami,"$2,400,000","2,631",$912,2008,"Christopher Jude PGutman, LLC","Daniel Hernandez Compass Florida, LLC"
2,2025-08-20,ECHO CONDO,3300 NE 188th St Unit UPH15 + ROOF TOP in Aven...,"$2,200,000","2,962",$743,2015,Uri Sefchovich The Keyes Company,Uri Sefchovich The Keyes Company
3,2025-08-22,OCEAN THREE CONDO,18911 Collins Ave #1104 in Sunny Isles Beach,"$1,585,000","1,903",$833,2003,Lana Levitina Coldwell Banker Realty,Oswaldo Sanchez Robleno Miami New Realty
4,2025-08-20,YACHT HARBOUR CONDO,2901 S Bayshore Dr Unit 13F in Miami,"$1,575,000","1,587",$992,1975,Allison Blumenthal Brown Harris Stevens,Monica Cardoso EXP Realty South LLC
5,2025-08-20,1010 BRICKELL CONDO,1010 Brickell Ave #3501 in Miami,"$1,575,000","1,753",$898,2017,David Siddons Douglas Elliman Shakira Sanchez ...,Laura Preuss Kuhne Douglas Elliman
6,2025-08-20,NINE ISLAND AVENUE CONDO,9 Island Ave #814 in Miami Beach,"$1,542,500","2,388",$646,1981,Carol Curtis First Pro International Rlty,Carol Curtis First Pro International Rlty
7,2025-08-22,MEI CONDO,5875 Collins Ave #1603 in Miami Beach,"$1,475,000","1,094","$1,348",2008,Armando Javier Jasal Int'l Realty Silvia River...,"Christopher King Compass Florida, LLC"
8,2025-08-19,PARAMOUNT MIAMI WORLDCENT,851 NE 1st Ave #4502 in Miami,"$1,465,000","2,164",$677,2019,Lisa Van Wagenen Brown Harris Stevens Katrina ...,Lisa Van Wagenen Brown Harris Stevens
9,2025-08-19,ATLANTIC I AT THE POINT C,21200 Point Pl #1802 in Aventura,"$1,300,000","2,733",$476,2004,Paola Wagener Charles Rutenberg Realty FTL,NON-MEMBER MLS AGENT NON MEMBER MLS


In [84]:
csv_date_string = today.strftime("%m_%d_%Y")

In [85]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [86]:
chart_df.columns

Index(['SOLD DATE', 'LOCATION', 'ADDRESS', 'PRICE', 'SQUARE FEET', 'PPSF',
       'YEAR BUILT', 'LISTING AGENTS', 'BUYING AGENTS'],
      dtype='object')